In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import requests

In [2]:
# EXTRACT ALL SONG TITLES ALONG WITH THEIR SINGERS/COMPOSERS VIA BEAUTIFULSOUP
# https://open.spotify.com/playlist/69HIaVaMTLoMYpUoQkgYdv?si=7a61a1d6ab994afd

def get_all_songs(playlist_url):
    
    # Manually create Chrome options
    chrome_options = webdriver.ChromeOptions()
    # Add any additional options if needed

    # Manually create the Chrome service
    chrome_service = webdriver.chrome.service.Service(ChromeDriverManager().install())

    # Pass options and service to Chrome constructor
    driver = webdriver.Chrome(options=chrome_options, service=chrome_service)

    driver.get(playlist_url)

    driver.implicitly_wait(10)

    # Get the updated page source with dynamic content
    html_content = driver.page_source

    driver.quit()

    data = BeautifulSoup(html_content,'html.parser')
    song_names = data.find_all('div', class_='Text__TextElement-sc-if376j-0 ksSRyh encore-text-body-medium t_yrXoUO3qGsJS4Y6iXX standalone-ellipsis-one-line')
    composers = data.find_all('div',class_='Text__TextElement-sc-if376j-0 gYdBJW encore-text-body-small')
    search_input = []
    for i in range(len(song_names)):
        temp = song_names[i].text
        temp+=" - "
    #     print(temp)
        temp_composers = composers[i].find_all('a')
        for j in temp_composers:
            temp+= j.text+", "
    #     print(temp)
        search_input.append(temp)
    return search_input

Yahan - Vasu Raina, 
Husn - Anuv Jain, 
Saaz-E-Dil - Akash, 
Meri Baaton Mein Tu - Anuv Jain, 
Saari Umar - Harish Budhwani, 
Iraaday - Rovalio, Abdul Hannan, 
kaise batayein tumhe? - Kanishk Rathore, 
Har Woh Pal - Aditya Satpathy, 
Fidaa - Shubhranshu Tiwari, 
Mulaqat - Prateek Kuhad, 
Ye Baarish - Darshan Raval, 
Nahin Milta - Bayaan, 


In [3]:
# SEARCH FOR EACH SONG IN YOUTUBE AND GET THERE URLS

def get_all_urls(search_input):

    # Manually create Chrome options
    chrome_options = webdriver.ChromeOptions()
    # Add any additional options if needed

    # Manually create the Chrome service
    chrome_service = webdriver.chrome.service.Service(ChromeDriverManager().install())

    # Pass options and service to Chrome constructor
    driver = webdriver.Chrome(options=chrome_options, service=chrome_service)

    driver.get('https://www.youtube.com/')

    wait = WebDriverWait(driver,100000)
    wait.until(EC.presence_of_element_located((By.XPATH,'//input[@name = "search_query"]')))
    search_box = driver.find_element(By.XPATH,'//input[@name = "search_query"]')
    yt_urls = []
    for i in search_input:
    #   IT WILL SEARCH FOR THE YOUTUBE URL OF THAT SONG
        search_box = driver.find_element(By.XPATH,'//input[@name = "search_query"]')
        search_box.clear()
        search_box.send_keys(i)
        search_box.submit()
    #   NOW WE WILL APPLY A WAIT TO GET THE YOUTUBE LINK
        wait = WebDriverWait(driver,100000)
        wait.until(EC.presence_of_element_located((By.ID,'video-title')))
        yt_link = driver.find_element(By.ID,'video-title')
        yt_link.click()
        wait = WebDriverWait(driver,100000)
        wait.until(EC.presence_of_element_located((By.XPATH,'//button[contains(@class,"ytp-mute-button ytp-button")]')))
    #     mute_button = driver.find_element(By.XPATH,'//button[contains(@class,"ytp-mute-button ytp-button")]')
    #     mute_button.click()
        yt_urls.append(driver.current_url)
        driver.get('https://www.youtube.com/')
    return yt_urls

In [6]:
# yt_urls = ['https://www.youtube.com/watch?v=4q7ml8ltKzc', 'https://www.youtube.com/watch?v=gJLVTKhTnog', 'https://www.youtube.com/watch?v=hDl17T5YliU', 'https://www.youtube.com/watch?v=hUORvCLETbI', 'https://www.youtube.com/watch?v=tV9HYDJAv9w', 'https://www.youtube.com/watch?v=Qwm6BSGrOq0', 'https://www.youtube.com/watch?v=N7Sfd7dRt1M', 'https://www.youtube.com/watch?v=wvN7lygwUTA', 'https://www.youtube.com/watch?v=08ImW54sCa8', 'https://www.youtube.com/watch?v=vzdZCm7cAqA', 'https://www.youtube.com/watch?v=RimXddCwM2I', 'https://www.youtube.com/watch?v=umsrQSA2iOM']

In [21]:
# DOWNLOADING SONGS

def download_songs(yt_urls):

    driver.get('https://ytmp3.nu/Q2rp/')
    count = 0
    for i in yt_urls:
        wait = WebDriverWait(driver,1000)
        wait.until(EC.presence_of_element_located((By.NAME,'url')))
        search_box = driver.find_element(By.NAME,'url')
        search_box.clear()
    #     print(search_box.get_attribute('outerHTML'))
        search_box.send_keys(i)
        search_box.submit()
        wait = WebDriverWait(driver,1000)
        wait.until(EC.presence_of_element_located((By.LINK_TEXT,'Download')))
        download_button = driver.find_element(By.LINK_TEXT,'Download')
        download_button.click()
        driver.implicitly_wait(10)
    #     print(download_button.get_attribute('outerHTML'))
        convert_next_button = driver.find_element(By.LINK_TEXT,'Convert next')
        convert_next_button.click()

In [ ]:
# SAMPLE INPUT - https://open.spotify.com/playlist/69HIaVaMTLoMYpUoQkgYdv?si=7a61a1d6ab994afd
print("--------WELCOME TO PLAYLIST-DOWNLOAD-CENTRE--------")
playlist_url = input()
search_input = get_all_songs(search_input)
yt_urls = get_all_urls()
download_songs(yt_urls)
print("DOWNLOADED SUCCESSFULLY")